In [1]:
import networkx as nx
from typing import List, Tuple, Union, Set


def find_links_n_hops_away(
    node_id: Union[str, int],
    graph: nx.Graph,
    num_hops: int
) -> List[Tuple[Union[str, int], Union[str, int]]]:
    """
    Find all edges (links) that are exactly N hops away from a given node.

    An edge is considered N hops away if at least one of its endpoints is
    exactly N hops from the source node.

    Parameters:
    -----------
    node_id : str or int
        The starting node ID/name
    graph : networkx.Graph
        The graph to search in (can be Graph, DiGraph, MultiGraph, etc.)
    num_hops : int
        The number of hops away to search for edges

    Returns:
    --------
    List[Tuple[str/int, str/int]]
        List of tuples where each tuple represents an edge (node1, node2)

    Example:
    --------
    >>> G = nx.Graph()
    >>> G.add_edges_from([('A', 'B'), ('B', 'C'), ('C', 'D'), ('D', 'E')])
    >>> find_links_n_hops_away('A', G, 2)
    [('B', 'C'), ('C', 'B')]  # Edge between B and C is 2 hops from A
    """

    # Validate inputs
    if node_id not in graph:
        raise ValueError(f"Node '{node_id}' not found in the graph")

    if num_hops < 0:
        raise ValueError("Number of hops must be non-negative")

    # Handle special case: 0 hops
    if num_hops == 0:
        # Return edges incident to the source node
        edges = []
        for neighbor in graph.neighbors(node_id):
            edges.append((node_id, neighbor))
        return edges

    # Find all nodes at exactly N hops using BFS
    nodes_at_n_hops = set()
    nodes_at_n_minus_1_hops = set()

    # BFS to find nodes at each distance
    visited = {node_id: 0}  # {node: distance_from_source}
    queue = [(node_id, 0)]  # (node, distance)

    while queue:
        current_node, distance = queue.pop(0)

        # Track nodes at N and N-1 hops
        if distance == num_hops:
            nodes_at_n_hops.add(current_node)
        elif distance == num_hops - 1:
            nodes_at_n_minus_1_hops.add(current_node)

        # Continue BFS only if we haven't reached N hops
        if distance < num_hops:
            for neighbor in graph.neighbors(current_node):
                if neighbor not in visited:
                    visited[neighbor] = distance + 1
                    queue.append((neighbor, distance + 1))

    # Find all edges that are exactly N hops away
    edges_n_hops_away = []

    # Strategy: An edge is N hops away if:
    # 1. Both endpoints are at N hops, OR
    # 2. One endpoint is at N hops and the other is at N-1 hops

    for u, v in graph.edges():
        u_dist = visited.get(u, float('inf'))
        v_dist = visited.get(v, float('inf'))

        # Check if this edge involves nodes at N hops
        if u_dist == num_hops or v_dist == num_hops:
            # Make sure at least one endpoint is exactly at N hops
            # and the edge connects nodes at distance N and N-1, or both at N
            if (u_dist == num_hops and v_dist == num_hops - 1) or \
               (v_dist == num_hops and u_dist == num_hops - 1) or \
               (u_dist == num_hops and v_dist == num_hops):
                edges_n_hops_away.append((u, v))

    return edges_n_hops_away


def find_links_n_hops_away_strict(
    node_id: Union[str, int],
    graph: nx.Graph,
    num_hops: int
) -> List[Tuple[Union[str, int], Union[str, int]]]:
    """
    Find all edges where BOTH endpoints are exactly N hops away from the source.

    This is a stricter interpretation where we only return edges between
    two nodes that are both exactly N hops from the source.

    Parameters:
    -----------
    node_id : str or int
        The starting node ID/name
    graph : networkx.Graph
        The graph to search in
    num_hops : int
        The number of hops away

    Returns:
    --------
    List[Tuple[str/int, str/int]]
        List of edges where both endpoints are N hops from source
    """

    if node_id not in graph:
        raise ValueError(f"Node '{node_id}' not found in the graph")

    if num_hops < 0:
        raise ValueError("Number of hops must be non-negative")

    # Find nodes at exactly N hops
    nodes_at_n_hops = set()
    visited = {node_id: 0}
    queue = [(node_id, 0)]

    while queue:
        current_node, distance = queue.pop(0)

        if distance == num_hops:
            nodes_at_n_hops.add(current_node)

        if distance < num_hops:
            for neighbor in graph.neighbors(current_node):
                if neighbor not in visited:
                    visited[neighbor] = distance + 1
                    queue.append((neighbor, distance + 1))

    # Find edges between nodes that are both at N hops
    edges_n_hops_away = []
    for u, v in graph.edges():
        if u in nodes_at_n_hops and v in nodes_at_n_hops:
            edges_n_hops_away.append((u, v))

    return edges_n_hops_away


# Example usage and test cases
if __name__ == "__main__":
    # Example 1: Simple linear graph
    print("Example 1: Linear Graph")
    G1 = nx.Graph()
    G1.add_edges_from([('A', 'B'), ('B', 'C'), ('C', 'D'), ('D', 'E')])

    print(f"Graph edges: {list(G1.edges())}")
    for hops in range(4):
        links = find_links_n_hops_away('A', G1, hops)
        print(f"Links {hops} hops from 'A': {links}")

    print("\n" + "="*50 + "\n")

    # Example 2: More complex graph with cycles
    print("Example 2: Graph with Cycle")
    G2 = nx.Graph()
    G2.add_edges_from([
        ('A', 'B'), ('A', 'C'),
        ('B', 'D'), ('C', 'D'),
        ('D', 'E'), ('E', 'F')
    ])

    print(f"Graph edges: {list(G2.edges())}")
    for hops in range(4):
        links = find_links_n_hops_away('A', G2, hops)
        print(f"Links {hops} hops from 'A': {links}")

    print("\n" + "="*50 + "\n")

    # Example 3: Comparing strict vs. regular version
    print("Example 3: Strict vs Regular")
    G3 = nx.Graph()
    G3.add_edges_from([
        (1, 2), (1, 3),
        (2, 4), (3, 4),
        (4, 5), (4, 6),
        (5, 6)
    ])

    print(f"Graph edges: {list(G3.edges())}")
    hops = 2
    regular_links = find_links_n_hops_away(1, G3, hops)
    strict_links = find_links_n_hops_away_strict(1, G3, hops)

    print(f"\nRegular version ({hops} hops from node 1): {regular_links}")
    print(f"Strict version ({hops} hops from node 1): {strict_links}")
    print("\nExplanation:")
    print("- Regular: Returns edges with at least one endpoint at N hops")
    print("- Strict: Returns only edges with BOTH endpoints at N hops")

Example 1: Linear Graph
Graph edges: [('A', 'B'), ('B', 'C'), ('C', 'D'), ('D', 'E')]
Links 0 hops from 'A': [('A', 'B')]
Links 1 hops from 'A': [('A', 'B')]
Links 2 hops from 'A': [('B', 'C')]
Links 3 hops from 'A': [('C', 'D')]


Example 2: Graph with Cycle
Graph edges: [('A', 'B'), ('A', 'C'), ('B', 'D'), ('C', 'D'), ('D', 'E'), ('E', 'F')]
Links 0 hops from 'A': [('A', 'B'), ('A', 'C')]
Links 1 hops from 'A': [('A', 'B'), ('A', 'C')]
Links 2 hops from 'A': [('B', 'D'), ('C', 'D')]
Links 3 hops from 'A': [('D', 'E')]


Example 3: Strict vs Regular
Graph edges: [(1, 2), (1, 3), (2, 4), (3, 4), (4, 5), (4, 6), (5, 6)]

Regular version (2 hops from node 1): [(2, 4), (3, 4)]
Strict version (2 hops from node 1): []

Explanation:
- Regular: Returns edges with at least one endpoint at N hops
- Strict: Returns only edges with BOTH endpoints at N hops
